<table style="width: 100%; border-style: none;">
<tr style="border-style: none">
<td style="border-style: none; width: 1%; font-size: 16px; text-align: left">Institut f&uuml;r Theoretische Physik<br /> Universit&auml;t zu K&ouml;ln</td>
<td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Prof. Dr. Simon Trebst<br />Jan Attig</td>
</tr>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computerphysik</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">&Uuml;bungsblatt 3</h1>
<hr>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">Sommersemester 2019</h3>

**Website:** [http://www.thp.uni-koeln.de/trebst/Lectures/2019-CompPhys.shtml](http://www.thp.uni-koeln.de/trebst/Lectures/2019-CompPhys.shtml)

**Abgabe der Bonusaufgabe**: Montag, 29.4.2019, **vor** der Vorlesung

**Besprechung**: 24.4.2019 - 26.4.2019


**Name**: <font color="red">Bitte geben Sie ihren Namen an</font>

**Matrikelnummer**: <font color="red">Bitte geben Sie ihre Matrikelnummer an</font>

<table style="border-style: none; width: 100%"><tr style="border-style: none;">
<td style="border-style: none; width: 1%; font-size: 20px; font-weight: bold; text-align: left">Aufgaben auf Übungsblatt 3</td>
<td style="border-style: none; width: 1%; font-size: 15px; text-align: right"></td></tr></table>

Auf Grund der Ostertage haben wir uns entschieden, auf diesem Übungszettel nur eine Aufgabe (sowie eine Bonusaufgabe) zu stellen damit Sie zusätzliche Zeit über die Tage und in den Übungsgruppen nächste Woche mit der Wiederholung der Programmier-Grundlagen (Blatt 1 und 2) verbringen können. Wir möchten auch an die Julia Resourcen auf der  [Kurs-Website](http://www.thp.uni-koeln.de/trebst/Lectures/2019-CompPhys.shtml) erinnern, die viele nützliche Tutorials enthalten.
- Rauschmittel [5 Punkte]
- **Zelluläre Automaten** [10 Bonus-Punkte]

<table style="border-style: none; width: 100%"><tr style="border-style: none;">
<td style="border-style: none; width: 1%; text-align: left; font-size: 20px; font-weight: bold">Zelluläre Automaten</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 15px">(10 Bonuspunkte)</td></tr></table>

In dieser *optionalen Aufgabe* betrachten wir die zeitlichen Entwicklung zweier **2D zellulärer Automaten**. Ein solcher zellulärer Automat ist ein diskretes System aus einzelnen Zellen, die in einem Quadratgitter angeordnet sind, und basierend auf ihrem eigenen Zustand und dem der umgegebenen Zellen in jedem Zeitschritt aktualisiert werden.

In dieser Aufgabe wollen wir uns auf zwei verschiedene Systeme konzentrieren. Zum einen behandeln wir einen *chemischen zellulären Automaten* mit drei Parametern $a, b, c \in [0, 1)$ und zum anderen das bekannte *Conway's Game of Life*.

Um Ihnen den Einstieg in diese Aufgabe zu erleichern, stellen wir Ihnen die nötigen Funktionen zur Seite, die Ihren Automaten animieren werden (basierend auf Ihrer Berechnung der Zustände). Beachten Sie jedoch bitte, dass diese Funktionen nicht in Julia Box funktionieren.

In [ ]:
# Useful Base Libraries
using LinearAlgebra
using Statistics

# Plotting Stuff
using PyPlot # plotting library
pygui(true)  # external window

# generieren der Figur
function generate_image(data)
    fig = figure(figsize=(6,6), frameon=false)
    image = imshow(data'[end:-1:1,:], interpolation="none")
    axis("off")
    fig.subplots_adjust(bottom=0, top=1, right=1, left=0)
    return image
end

# Updaten der Figur
function update_image(image, data)
    image.set_array(data'[end:-1:1,:])
    pause(0.001)
end

Um Ihnen einen einfachen Einstieg in die Programmierung von zellulären Automaten zu geben, stellen wir Ihnen zusätzlich eine Funktion `neighbors` bereit, die die Summe der Nachbarn um eine Stelle `i,j` in einem gegebenen Array berechnet.

In [ ]:
# Summe der Nachbarn um (i,j) im Array data berechnen
function neighbors(data, i, j)
    if i == size(data,1)
        ip1 = 1
        im1 = i-1
    elseif i == 1
        ip1 = i+1
        im1 = size(data,1)
    else
        ip1 = i+1      
        im1 = i-1 
    end
    if j == size(data,2)
        jp1 = 1
        jm1 = j-1
    elseif j == 1
        jp1 = j+1
        jm1 = size(data,2)
    else
        jp1 = j+1
        jm1 = j-1
    end
    return  data[ip1, j] +
            data[im1, j] +
            data[ip1, jp1] +
            data[im1, jp1] +
            data[ip1, jm1] +
            data[im1, jm1] +
            data[i, jp1] +
            data[i, jm1]
end

### 1. Chemischer zellulärer Automat

Bei diesem Automaten werden die Zellen durch drei Parameter $a, b, c \in [0, 1)$ beschrieben werden, welche die Konzentration dreier Chemikalien beschreiben sollen. Das Aktualisieren der Werte soll chemische Reaktionen zwischen diesen Substanzen beschreiben. Zu Anfang seien die drei Konzentrationen zufällig aus dem möglichen Wertebereich gewählt.

Der zelluläre Automat funktioniert nun nach den folgenden Regeln:
* Für jede Zelle wird der durchschnittliche Wert $\overline{a}, \overline{b}, \overline{c}$ als Mittelwert der acht umgebenden Zellen sowie des Werts der Zelle selbst berechnet.
* Der neue Zustand ist dann gegeben durch

$\quad a = \min\left(1.0,\; \overline{a}\cdot\left(1. + \overline{b} - \overline{c}\right)\right)$

$\quad b = \min\left(1.0,\; \overline{b}\cdot \left(1. + \overline{c} - \overline{a}\right)\right)$

$\quad c = \min\left(1.0,\; \overline{c}\cdot \left(1. + \overline{a} - \overline{b}\right)\right)$

Ihre Aufgabe ist es das Programm-Skelett in den nachfolgenen Zellen zu vervollständigen und um das Update der Automaten-Zellen zu erweitern. Wie Sie erkennen können, wird der Automat bereits visualisiert indem eine der drei Konzentrationen dargestellt und in jedem Schritt aktualisiert wird. 

Es kann hilfreich sein, die Funktion `neighbors` zu benutzen, die oben bereits definiert ist. Diese erzeugt die Summe der umliegenden Nachbarn in einem gegebenen Array an einer gegebenen Position.

Beachten Sie schließlich, dass alle Zellen *gleichzeitig* in den neuen Zustand übergehen, d.h. die Berechnung der Mittelwerte darf ausschließlich mit den Werten des vorherigen Iterationsschritts passieren und nicht mit denen der schon partiell veränderten Matrix.


In [ ]:
# Systemgroesse
L = 128

# Die Werte a,b,c des Automaten als 3 Arrays der Groesse LxL
automaton_a = rand(L, L)
automaton_b = rand(L, L)
automaton_c = rand(L, L)

# Arrays fuer die Mittelwerte
automaton_mean_a = zeros(L, L)
automaton_mean_b = zeros(L, L)
automaton_mean_c = zeros(L, L)

# Erstes darstellen des Bildes
image = generate_image(automaton_a)


# Fuehre Schritte aus
for step in 1:200
    
    ############################################
    # HIER SOLLTE IHR CODE STEHEN
    # Berechne die Mittelwerte    
    # Berechne die neuen Automatenzustaende
    ############################################

    # Aktualisiere das Bild
    update_image(image, automaton_a)
end

### 2. Conway's Game of Life

Im Folgenden wollen wir nun einen weiteren zellulären Automaten betrachten, der als [Conway's Game of Life](https://de.wikipedia.org/wiki/Conways_Spiel_des_Lebens) bekannt wurde. Die Grundlage dieses Automaten sind erneut Zellen, die in einem Quadratgitter angeordnet sind. Im Unterschied zum biologischen Automaten können diese jedoch nur zwischen zwei Zuständen wechseln, die wir im Folgenden als tot (`0`) und lebendig (`1`) bezeichnen wollen.

In jedem Iterationsschritt wird nun für jede Zelle geprüft, ob sie lebt oder tot ist und wie viele lebendige Nachbarn sie besitzt. Daraus wird entwickelt, wie sich die Zelle im nächsten Iterationsschritt verhält:
- Eine Zelle mit weniger als 2 lebendigen Nachbarn ist sicher im nächsten Schritt tot
- Eine tote Zelle mit exakt 3 lebendigen Nachbarn wird im nächsten Schritt geboren
- Eine lebeindige Zelle mit mehr als 3 lebendigen Nachbarn stirbt in der nächsten Generation an Überbevölkerung

Ihre Aufgabe ist die Implementierung des Automaten basierend auf dieser kurzen Beschreibung sowie dem Code aus dem vorherigen Aufgabenteil. Testen Sie Ihre Implementierung indem Sie ein zufällig besetztes Array generieren und dieses als Startkonfiguration benutzen.

In [ ]:
# Systemgroesse
L = 128

############################################
# HIER SOLLTE IHR CODE STEHEN
# Das Array das die Zellen enthaelt erstellen
# Array fuer Nachbaranzahlen erstellen
# Erstes darstellen des Bildes
# Schleife ueber Schritte
    # Berechne die Mittelwerte    
    # Berechne die neuen Automatenzustaende
    # Bild aktualisieren
############################################

Wie Sie sicherlich erkennen konnten, entwickelt sich die zufällige Konfiguration in vielerlei Hinsicht. Es gibt kleine Strukturen und Cluster die scheinbar statisch scheinen, sowie Elemente, die kurze Perioden durchlaufen (sogenannte Pulsare). Die Menge an verschiedene Konfigurationen und Mustern ist gigantisch und kann im [Game of Life Wiki](http://www.conwaylife.com/wiki/Main_Page) nachgelesen werden. Wenn sie dort verschiedenen Konfigurationen, wie die [Gosper Gleiterkanone](http://www.conwaylife.com/wiki/Gosper_glider_gun) o.Ä. betrachten, können Sie diese auch in einem sogenannten [RLE](http://www.conwaylife.com/wiki/RLE) Format direkt kopieren.

Probieren Sie Ihren Code um eine Funktion zu erweitern, die aus einen RLE String in ihre Simulation einlesen kann. Welches Muster finden Sie am interessantesten?

Hinweis: Sie mögen das String Macro `raw"..."` hier nützlich finden, da es Strings mit enthaltenen `$` darstellen kann. Z.B. erzeugt `raw"ob$b"` wirklich den entsprechenden String und will nicht den Buchstaben `b` interpretieren.